In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence,RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
import os

In [2]:
load_dotenv()

True

In [5]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [7]:
parser = StrOutputParser()

In [23]:
#  Runnable Sequence -> A sequence of chain of Runnables that execute  each step one after another. Output of one step is the input of the next step.

prompt1 = PromptTemplate(
    template='Write a term specific about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Explain the following term - {text}',
    input_variables=['text']
)

chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)
print(chain.invoke({'topic':'Cricket'}))

Your explanation of "Silly Point" is accurate and well-described. Here's a breakdown of why it's a good explanation and some minor points to emphasize:

*   **Position:** It clearly states it's a fielding position on the leg side for a right-handed batsman (or off-side for a left-handed batsman).  The key is understanding it's *very close* to the batsman.

*   **Risk:** You emphasize the inherent danger. The fielder is extremely close and has little time to react to a powerfully struck ball.

*   **Etymology:** The explanation of the name's origin is also correct. The "silly" part refers to the perceived foolishness or risk associated with fielding in such a dangerous spot.

**Key takeaways and potential additions for extra clarity:**

*   **Purpose:** Briefly mention the purpose. Silly Point is generally positioned to catch a mis-timed flick, a glove-edge, or a bat-pad catch. It's a position typically used when the bowler is trying to cramp the batsman for room and induce a mistake.



In [ ]:
# Runnable Parallel -> Run multiple sequential chains at the same time -> Output is a dictionary

prompt1 = PromptTemplate(
    template='Geneate a linkedin post for the following {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a twitter post for the following {topic}',
    input_variables=['topic']
)

chain = RunnableParallel(
    {
        'linkedin': RunnableSequence(prompt1,model,parser),
        'twitter': RunnableSequence(prompt2,model,parser)
    }
)

result = chain.invoke({'topic':"Generative AI"})

print(result['linkedin'])
print(result['twitter'])

Okay, here are a few LinkedIn post options for Generative AI, ranging from beginner-friendly to more thought-provoking.  Choose the one that best suits your audience and your personal brand:

**Option 1:  Beginner-Friendly & Enthusiastic**

**Headline:** Generative AI is Changing the Game! 🚀

**Body:**

Just diving into the world of Generative AI, and I'm blown away by the possibilities! From creating stunning images to writing compelling content, it's a game-changer for creativity and productivity.

What are some of the most exciting applications of Generative AI you've seen? Let's discuss! #GenerativeAI #AI #ArtificialIntelligence #Innovation #FutureofWork

**Option 2:  Industry-Specific Focus (Example: Marketing)**

**Headline:** Generative AI: The Future of Marketing is Here 🤖

**Body:**

As marketers, we're always looking for ways to create more engaging content and streamline our workflows. Generative AI is offering incredible opportunities to do just that!

Imagine:
*   Generati

In [25]:
# Runnable Passthrough -> Simply returns input as output without modifying it

prompt1 = PromptTemplate(
    template='Write a term specific about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Explain the following term - {text}',
    input_variables=['text']
)

chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'topic': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

final_chain = RunnableSequence(chain, parallel_chain)

result = final_chain.invoke({'topic':'cricket'})

print(result['topic'])
print(result['explanation'])

**Googly**

A googly is a type of delivery bowled by a right-arm leg spin bowler. It spins in the opposite direction to a normal leg break, turning from off to leg (right to left from the bowler's perspective). This makes it particularly deceptive, as it can catch batsmen off guard and lead to wickets.
Okay, here's a breakdown of the term "Googly" in cricket, based on your definition:

**What a Googly Is:**

*   **A Deceptive Leg Spin Delivery:** A googly is a special type of ball bowled by a leg-spin bowler (a right-handed bowler who usually makes the ball spin from leg to off, i.e., from the batsman's leg side to the off side).

*   **Reverse Spin:**  The key is that the googly spins *in the opposite direction* compared to the bowler's standard leg break. Instead of spinning from leg to off, it spins from off to leg (right to left from the bowler's perspective).

*   **Deception is Key:** This reverse spin is what makes it so effective. The bowler's action might look very similar to 

In [29]:
# RunnableLambda -> Wraps any Python function as runnable

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

chain = RunnableSequence(prompt, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    # 'word_count': RunnableLambda(word_count)
    'word_count': RunnableLambda(lambda x: len(x.split()))
})

final_chain = RunnableSequence(chain, parallel_chain)

result = final_chain.invoke({'topic':'Apple'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

Why did the Apple product get a flat tire?

Because it had no Windows! 
 word count - 14


In [31]:
# Runnable Branch -> route the input to different branches based on the conditions

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | model | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'IPL 2025'}))

The IPL 2025 season saw the Mumbai Indians (MI) win their 7th title, defeating the Gujarat Titans (GT) in the final. Shivam Dube (CSK) won the Orange Cap, and Jasprit Bumrah (MI) took home both the Purple Cap and MVP awards. Sai Sudharsan (GT) was named Emerging Player. MI and GT were the top teams, while RCB struggled again. The season highlighted trends like the increased use of data analytics, the importance of all-rounders, aggressive batting, and spin bowling. Hypothetical controversies included umpiring errors and code of conduct violations. The IPL 2025 continued to have a significant economic impact, developed cricket talent, expanded its global reach, and engaged fans effectively. The report emphasizes the need to address issues like umpiring and over rates for the league's future success.


In [32]:
# LCEL -> Langchain Expression Language
# Function chaining: chain1 | chain2 | chain3
# Multiple outputs: output in dictionary